In [1]:
from pathlib import Path
import json
import re
from tqdm import tqdm

settings = json.loads(Path('../../../settings.json').read_text())preprocessed_data_path = Path(settings['preprocessed_data_path'])data_path = Path('.').resolve()data_name = data_path.namesource_name = data_path.parent.namepreprocessed_dir = preprocessed_data_path/source_name/data_namesource_data_dir = preprocessed_dir/'preprocessed'
splits = ['train', 'valid']
tasks = ['LM(한국어)', '질문 생성', '문서 내용 QA']
task_data_dir = preprocessed_dir/'preprocessed_task'
task_data_dir.mkdir(exist_ok=True)
for task in tasks:
    task_path = task_data_dir/task
    task_path.mkdir(exist_ok=True)
    
#### prepare for task preprocess



#### prepare for task preprocess end

#### task preprocess
for split in splits:
    source_data_dir_split = source_data_dir/split
    task_files = [(task_data_dir/task/f'{split}.jsonl').open('w', encoding='utf-8') for task in tasks]
    for source_data in tqdm(list(source_data_dir_split.iterdir()), desc=split):
        source_data = source_data.open()
        for line in source_data.readlines():
            line = json.loads(line)
            
            #### data preprocess
            title = line['title']
            content_id = line['content_id']
            text = '\n'.join([p['context'] for p in line['paragraphs']])
            qa_dict = {}
            for paragraph in line['paragraphs']:
                for qa in paragraph['qas']:
                    qid = qa['id']
                    question = qa['question']
                    answer = qa['answers'][0]['text']
                    is_impossible = qa['is_impossible']
                    qa_dict[qid] = {'question': question, 'is_impossible': is_impossible, 'answer': answer}
            #### data preprocess end 
            
            #### LM(한국어)
            data = {'text': None}
            ## preprocess data from line
            text = {'title': title, 'content': text}
            text = json.dumps(text, ensure_ascii=False, indent=4)
            data['text'] = text
            ## preprocess data from line end
            task_files[0].write(json.dumps(data, ensure_ascii=False)+'\n')
            # print(json.dumps(data, indent=4, ensure_ascii=False))
            #### LM(한국어) end


            #### 질문 생성
            data = {'input': {}, 'output': {}}
            ## preprocess data from line
            data['input'] = {'title': title, 'text': text}
            data['input']['answers'] = [{'answer':qa['answer'], 'is_impossible':qa['is_impossible']} for qa in qa_dict.values()]
            data['output']['questions'] = [{'question':qa['question']} for qa in qa_dict.values()]
            ## preprocess data from line end
            task_files[1].write(json.dumps(data, ensure_ascii=False)+'\n')
            # print(json.dumps(data, indent=4, ensure_ascii=False))
            #### 질문 생성 end


            #### 문서 내용 QA
            data = {'input': None, 'output': None}
            ## preprocess data from line
            data['input'] = {'title': title, 'text': text}
            data['input']['questions'] = [qa['question'] for qa in qa_dict.values()]
            data['output'] = {'answers:': [qa['answer'] if not qa['is_impossible'] else '알 수 없음.' for qa in qa_dict.values()]}
            ## preprocess data from line end
            task_files[2].write(json.dumps(data, ensure_ascii=False)+'\n')
            # print(json.dumps(data, indent=4, ensure_ascii=False))
            #### 문서 내용 QA end


            
    #         break
    #     break
    # break

    for path in task_files:
        path.close()      
        
#### task preprocess end      

valid: 100%|██████████| 1/1 [00:00<00:00,  2.02it/s]
